<a href="https://colab.research.google.com/github/nehadubey1205/NLP-college/blob/main/BullySentBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import required libraries
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
data = pd.read_csv('/content/outputbullySent.csv')
data.tail(2)

,Tweets,Processed_Tweets,Bully_Label,Sentiment_label,Emotion_label,text
6082,chal be bhadwe sanghi.. teri maa ki choot me g...,chal be bhadwe sanghi teri maa ki choot me gho...,1,negative,disgust,Let's go crazy with your mother's pussy.
6083,"इश्क़ में थे जब,\nवादा हमारा,\nदावा भी हमारा,\...",इश्क़ में थे जबवादा हमारादावा भी हमाराधोखा उनक...,0,neutral,sadness,"When we were in love, our claim was also our b..."


In [10]:

from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split


In [11]:
data.head(2)

,Tweets,Processed_Tweets,Bully_Label,Sentiment_label,Emotion_label,text
0,teri maa ki chut me shivsena ka lund gaya kya,teri maa ki chut me shivsena ka lund gaya kya,1,negative,anger,Did Shiv Sena's dick go in your mother's pussy?
1,han muje pata hai teri maa ka ke gufa me jati thi,han muje pata hai teri maa ka ke gufa me jati thi,1,negative,others,yes i know your mother used to go to the cave


In [12]:
# shift column 'Name' to first position
first_column = data.pop('text')
  
# insert column using insert(position,column_name,
# first_column) function
data.insert(0, 'text', first_column)
  
print()
print("After Shifting column to first position")
display(data)


After Shifting column to first position


,text,Tweets,Processed_Tweets,Bully_Label,Sentiment_label,Emotion_label
0,Did Shiv Sena's dick go in your mother's pussy?,teri maa ki chut me shivsena ka lund gaya kya,teri maa ki chut me shivsena ka lund gaya kya,1,negative,anger
1,yes i know your mother used to go to the cave,han muje pata hai teri maa ka ke gufa me jati thi,han muje pata hai teri maa ka ke gufa me jati thi,1,negative,others
2,"Where did you come from brother, you were sitt...",tum kaha se aa gye bhai tumlog toh saree pehen...,tum kaha se aa gye bhai tumlog toh saree pehen...,0,neutral,others
3,koi moti si ki best besan ka or chane ki dal k...,koi moti🙊kisi ki besti😁 baisn ka or channey ki...,koi motikisi ki besti baisn ka or channey ki d...,1,negative,disgust
4,Rahul Kanwal will dance naked on the road today,rahul kanwal aaj nanga hoke road per nachega 😂,rahul kanwal aaj nanga hoke road per nachega,1,negative,disgust
...,...,...,...,...,...,...
6079,dali sali kayo fhshdhdhdhd no need to follow m...,dali sali kayo fhshdhdhdhd no need to follow m...,dali sali kayo fhshdhdhdhd no need to follow m...,0,neutral,others
6080,Tum rajput pehle mughalon ka bed hot krti thi ...,@anjalisingh_inc tum rajput pehle mugalon ka b...,tum rajput pehle mugalon ka bistar garm krti t...,1,negative,disgust
6081,"On the occasion of circumcision, tell me one t...",khatna pe to bol diya ek baat bata upar wala t...,khatna pe to bol diya ek baat bata upar wala t...,1,negative,others
6082,Let's go crazy with your mother's pussy.,chal be bhadwe sanghi.. teri maa ki choot me g...,chal be bhadwe sanghi teri maa ki choot me gho...,1,negative,disgust


CLEANING PROCESS

In [13]:
# clean the dataset

import re



# remove unnecessary columns
data = data.drop(['Tweets','Processed_Tweets'], axis=1)

# handle missing values
data = data.fillna('')

In [14]:
# remove duplicates
data = data.drop_duplicates()

# remove stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:

stop_words = set(nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('hinglish'))

In [17]:
def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(words)

data['text'] = data['text'].apply(remove_stopwords)


In [18]:
# remove special characters and numbers
def remove_special_characters(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\d+', '', text)
    return text
    

In [19]:
data['text'] = data['text'].apply(remove_special_characters)

In [20]:
# stemming or lemmatization
stemmer = nltk.stem.SnowballStemmer('english')

def stem_text(text):
    words = text.split()
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

data['text'] = data['text'].apply(stem_text)

# save the cleaned dataset
data.to_csv('cleaned_hinglish_dataset.csv', index=False)


In [32]:
data = pd.read_csv('/content/cleaned_hinglish_dataset.csv')

In [25]:
# split the dataset into training and validation sets
#X_train, X_val, y_train, y_val = train_test_split(data['text'], data['Bully_Label'], test_size=0.2, random_state=42)

# load the BERT tokenizer and model
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
#model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [28]:
import torch

In [33]:
from transformers import BertTokenizer, BertForSequenceClassification
X_train, X_val, y_train, y_val = train_test_split(data['text'], data['Bully_Label'], test_size=0.2, random_state=42)
# Load the pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

# Load the training and validation dataset


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [35]:
data.head(2)

,text,Bully_Label,Sentiment_label,Emotion_label
0,shiv sena dick mother pussi,1,negative,anger
1,mother cave,1,negative,others


In [37]:
data.isnull().sum()

text               17
Bully_Label         0
Sentiment_label     0
Emotion_label       0
dtype: int64

In [38]:


data = data.dropna(axis=0, subset=['text'])

In [39]:
data.isnull().sum()

text               0
Bully_Label        0
Sentiment_label    0
Emotion_label      0
dtype: int64

BIGRAM WILL BE USEFUL FOR COMIBE TWO WORDS AND WILL HELP TO FIND OUT BAD WORD LIKE: mother f###

In [40]:

from sklearn.feature_extraction.text import CountVectorizer


# ...

# define the bigram vectorizer
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2), analyzer='char')

# fit the bigram vectorizer on the text data
bigram_vectorizer.fit(data['text'])

# transform the text data using the bigram vectorizer
bigram_features = bigram_vectorizer.transform(data['text'])

# create a dataframe of the bigram features
bigram_df = pd.DataFrame(bigram_features.toarray(), columns=bigram_vectorizer.get_feature_names())

# concatenate the bigram features with the original dataframe
data = pd.concat([data, bigram_df], axis=1)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [41]:
data.head(2)

,text,Bully_Label,Sentiment_label,Emotion_label,a,b,c,d,e,f,...,zl,zm,zo,zr,zs,zt,zu,zw,zy,zz
0,shiv sena dick mother pussi,1.0,negative,anger,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,mother cave,1.0,negative,others,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# tokenize the text data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True)

ValueError: ignored

In [ ]:
train_data = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True)
valid_data = torch.utils.data.DataLoader(dataset_valid, batch_size=32, shuffle=False)

In [24]:
# tokenize the text data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True)

In [ ]:
''' import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

# Load the training and validation dataset
train_data = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=True)
valid_data = torch.utils.data.DataLoader(dataset_valid, batch_size=32, shuffle=False)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Train the model
for epoch in range(5):
    model.train()
    for batch in train_data:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in valid_data:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, predicted = torch.max(outputs[0], 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Epoch [{}/{}], Accuracy: {:.2f}%'
              .format(epoch+1, 5, 100 * correct / total))

# Make predictions on new data
text = "You are so ugly"
inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
outputs = model(inputs['input_ids'], inputs['attention_mask'])
_, predicted = torch.max(outputs[0], 1)
print("Predicted class:", predicted.item())

'''

In [23]:
# convert the tokenized data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), torch.tensor(y_train.values))
val_dataset = torch.utils.data.TensorDataset(torch.tensor(val_encodings['input_ids']), torch.tensor(val_encodings['attention_mask']), torch.tensor(y_val.values))

# define the training parameters
optimizer = AdamW(model.parameters(), lr=1e-5)
epochs = 3
batch_size = 16

NameError: ignored

In [ ]:


# define the training and validation dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

# train the model
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        val_loss, val_accuracy = 0, 0
        for batch in val_loader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch


In [ ]:

# ...

# split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

In [ ]:


# tokenize the text data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True)

# convert the tokenized data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), torch.tensor(y_train.values))
val_dataset = torch.utils.data.TensorDataset(torch.tensor(val_encodings['input_ids']), torch.tensor(val_encodings['attention_mask']), torch.tensor(y_val.values))

# define the training parameters
optimizer = AdamW(model.parameters(), lr=1e-5)
epochs = 3
batch_size = 16

# define the training and validation dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

# train the model
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        val_loss, val_accuracy = 0, 0
        for batch in val_loader:
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch


In [4]:
first_column = data.pop('text')
  
# insert column using insert(position,column_name,
# first_column) function
data.insert(0, 'text', first_column)

In [5]:
print()
print("After Shifting column to first position")


After Shifting column to first position


In [6]:
display(data.tail(5))

,text,Tweets,Processed_Tweets,Bully_Label,Sentiment_label,Emotion_label
6079,dali sali kayo fhshdhdhdhd no need to follow m...,dali sali kayo fhshdhdhdhd no need to follow m...,dali sali kayo fhshdhdhdhd no need to follow m...,0,neutral,others
6080,Tum rajput pehle mughalon ka bed hot krti thi ...,@anjalisingh_inc tum rajput pehle mugalon ka b...,tum rajput pehle mugalon ka bistar garm krti t...,1,negative,disgust
6081,"On the occasion of circumcision, tell me one t...",khatna pe to bol diya ek baat bata upar wala t...,khatna pe to bol diya ek baat bata upar wala t...,1,negative,others
6082,Let's go crazy with your mother's pussy.,chal be bhadwe sanghi.. teri maa ki choot me g...,chal be bhadwe sanghi teri maa ki choot me gho...,1,negative,disgust
6083,"When we were in love, our claim was also our b...","इश्क़ में थे जब,\nवादा हमारा,\nदावा भी हमारा,\...",इश्क़ में थे जबवादा हमारादावा भी हमाराधोखा उनक...,0,neutral,sadness


In [7]:
# Remove irrelevant columns and clean the text
data = data.drop(['Tweets', 'Processed_Tweets'], axis=1)

In [8]:
data.tail(2)

,text,Bully_Label,Sentiment_label,Emotion_label
6082,Let's go crazy with your mother's pussy.,1,negative,disgust
6083,"When we were in love, our claim was also our b...",0,neutral,sadness


In [9]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.2 MB/s eta 0:00:00


In [10]:
import torch
import transformers as ppb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
# Load the BERT model and tokenizer
model_class = ppb.BertModel
tokenizer_class = ppb.BertTokenizer
pretrained_weights = 'bert-base-multilingual-cased'
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
